# thoughts

这个本质上是个用过去的数据预测未来第15天的事情。


# base

## base train

In [10]:
train_path = "/Users/dongjian/data/ueq_50wan_train__100fea"
test_path = "/Users/dongjian/data/ueq_10wan_test__100fea"
out_path = "/Users/dongjian/data/ueq_10wan_out"
label = "reorder"

In [6]:
to_drop = [
    "uuid", "poi_id", "wm_poi_id", "reorder", "dt", "order_time", "user_id", "wk", 'mo', 'city_id', "is_discount_fee",
    "is_donation", "shipping_fee", "id", "total", "original_price", "longitude", "latitude", "timescope_order_cnt",
    "timescope_total_price", "timescope_original_price","user_poi_order_decay"
    # ,"user_reorder_rate","user_reorder_count"
]

# drop 85
# fea_drop =  [u'order_cnt_5hr', u'order_cnt_3hr', u'order_cnt_2hr', u'order_cnt_4hr', u'order_cnt_8hr', u'original_price_7day', u'order_cnt_6hr', u'comment_2star', u'service_fee_rate_7day', u'order_cnt_7day', u'order_cnt_1hr', u'order_cnt_23hr', u'order_cnt_increase', u'order_cnt_7hr', u'order_cnt_9hr', u'order_cnt_24hr', u'order_cnt_15hr', u'comment_1star', u'order_cnt_22hr', u'order_cnt_19hr', u'order_cnt_12hr', u'order_cnt_14hr', u'pv_cxr_1day', u'order_cnt_17hr', u'order_cnt_21hr', u'order_cnt_18hr', u'order_cnt_16hr', u'pic_comment_cnt', u'uv_ctr_1day', u'avg_food_comment_score', u'uv_cvr_15day', u'comment_4star', u'month_original_price', u'user_reorder_count', u'click_tag_pref_3day', u'dp_score', u'month_order_cnt', u'month_total_price', u'uv_cxr_7day', u'op_time_30day', u'click_tag_pref_30day', u'click_tag_pref_15day', u'pv_ctr_7day', u'uv_cvr_7day', u'distance_30day', u'order_cnt_13hr', u'pv_ctr_15day', u'uv_cvr_1day', u'uv_ctr_15day', u'service_fee_7day', u'pv_ctr_1day', u'discount_rate_all_customer_7day', u'food_comment_rate', u'pos_delivery_comment_rate', u'avg_delivery_comment_score', u'dp_avg_price', u'pv_cvr_1day', u'order_cnt_20hr', u'pos_comment_rate', u'click_tag_pref_7day', u'avg_price_month', u'area_30day', u'food_comment_cnt', u'poi_one_bought',
#              u'comment_3star',
#              u'uv_ctr_7day', u'uv_cxr_1day', u'service_fee_1day',
#              u'comment_5star',
#              u'comment_5star_rate', u'pic_comment_rate', u'avg_comment_score', u'order_cnt_11hr',
#              u'total_price_1day',
#              # "order_cnt_1day",
#              u'original_price_1day', u'discount_rate_new_customer_7day', u'uv_cxr_15day', u'order_cnt_14day', u'tag_id', u'neg_delivery_comment_rate', u'pv_cxr_7day', u'comment_uv', u'pv_cvr_7day', u'poi_two_bought', u'poi_reorder_counts']
# features = list((set(basic_features) | set(advanced_features)) - set(to_drop) - set(fea_drop))
#drop 45
fea_drop =["tag_id"]

In [11]:
train_data = deco_data(pd.read_csv(train_path, sep="\t", skiprows=1))
test_data = deco_data(pd.read_csv(test_path, sep="\t", skiprows=1))
print train_data.head(5)
print train_data.dtypes
features = list(
        set(train_data.columns.tolist()[1:]) - set(
                to_drop) - set(fea_drop))

print "len feas {}, feas {}".format(len(features), features)
# features = best_feas
# train_data.fillna(-999, inplace=True)
# test_data.fillna(-999, inplace=True)

labels = train_data[label].values.astype(np.float32).flatten()
labels_val = test_data[[label]].values.astype(np.float32).flatten()
lgb_train = lgb.Dataset(train_data[features], labels)
lgb_eval = lgb.Dataset(test_data[features], labels_val, reference=lgb_train)

print('Start training...')

params = {'num_leaves': 100, 'task': 'train', 'verbose': 1, 'learning_rate': 0.03, 'nthread': 8,
          'min_data_in_leaf': 150, 'objective': 'binary', 'boosting_type': 'gbdt', 'metric': ['logloss', 'auc'],
          'feature_fraction': 0.1, "bagging_fraction": 0.7, "bagging_freq": 5}

gbm = lgb.train(params,
                lgb_train,
                # fobj=huber_approx_obj,
                valid_sets=[lgb_eval],
                num_boost_round=500,
                early_stopping_rounds=30,
                verbose_eval=10)

df = pd.DataFrame({'feature': gbm.feature_name(), 'importances': gbm.feature_importance()})
df['fscore'] = df['importances'] / df['importances'].sum()

# print(df)
print(df.sort_values('importances', ascending=False).to_string(index=False))
print  "-" * 20 + " features" + "-" * 20
print [x.strip() for x in df.sort_values('importances', ascending=False).feature.to_string(index=False).split("\n")]
# print(df[df['fscore']<=0.005].feature.to_string(index=False))

test_data['prd'] = gbm.predict(test_data[features])
import os

# ax = lgb.plot_tree(gbm,tree_index=8, figsize=(20, 8), show_info=['split_gain'])
plt.show()
test_data.to_csv(out_path)

                   uuid  wm_poi_id  shipping_fee  order_time  total  \
0  -1591401001296264364    2759672           3.0  1505535257   53.3   
1  -4458817782016207280     361002           5.0  1505565650  106.8   
2  -5007896300539974560     282856           4.0  1505524222   17.0   
3  -5417409195447726191      93777           3.0  1505522960   26.5   
4  -6994174706008770126    2388058           2.5  1505555372   27.5   

   original_price  longitude  latitude  reorder  poi_reorder_rate    ...     \
0            95.0  118808205  32079721        0          0.350000    ...      
1           132.0  118716790  32003749        0          0.172566    ...      
2            31.0  117375404  39179274        0          0.144000    ...      
3            40.5  121479080  31287540        1          0.364004    ...      
4            29.5  106530550  29504910        0          0.341573    ...      

   click_tag_pref_30day  submit_tag_pref_3day  submit_tag_pref_7day  \
0                   0.0    

## new sample

### data read 

In [88]:
import gc
gc.collect()

177

In [12]:
inp_path = '~/data/'
data = pd.read_csv(inp_path+'ueq_input',skiprows=range(1,3),sep='\t',nrows=10**7,dtype={'dt':'string'})

In [13]:
data.columns = map(lambda x: x.replace("waimai_ad_feature_reorder_v11.", ""), data.columns.tolist())

In [14]:
data['dt'] = lookup(data['dt'])

In [15]:
data = data.set_index(data.dt)

In [16]:
np.unique(data['dt'])

array(['2017-10-15T00:00:00.000000000', '2017-10-16T00:00:00.000000000',
       '2017-10-17T00:00:00.000000000', '2017-10-18T00:00:00.000000000',
       '2017-10-19T00:00:00.000000000', '2017-10-20T00:00:00.000000000',
       '2017-10-21T00:00:00.000000000', '2017-10-22T00:00:00.000000000',
       '2017-10-23T00:00:00.000000000', '2017-10-24T00:00:00.000000000',
       '2017-10-25T00:00:00.000000000', '2017-10-26T00:00:00.000000000',
       '2017-10-27T00:00:00.000000000', '2017-10-28T00:00:00.000000000',
       '2017-10-29T00:00:00.000000000', '2017-10-30T00:00:00.000000000',
       '2017-10-31T00:00:00.000000000', '2017-11-01T00:00:00.000000000',
       '2017-11-02T00:00:00.000000000', '2017-11-03T00:00:00.000000000',
       '2017-11-04T00:00:00.000000000', '2017-11-05T00:00:00.000000000',
       '2017-11-06T00:00:00.000000000', '2017-11-07T00:00:00.000000000',
       '2017-11-08T00:00:00.000000000', '2017-11-09T00:00:00.000000000',
       '2017-11-10T00:00:00.000000000', '2017-11-11

In [17]:
pd.date_range('20171116',periods=10)

DatetimeIndex(['2017-11-16', '2017-11-17', '2017-11-18', '2017-11-19',
               '2017-11-20', '2017-11-21', '2017-11-22', '2017-11-23',
               '2017-11-24', '2017-11-25'],
              dtype='datetime64[ns]', freq='D')

In [18]:
train_date = pd.date_range('20171116',periods=10)
test_date = pd.date_range('20171210',periods=5)

In [19]:
train = data.loc[train_date,:]
test = data.loc[test_date,:]

In [20]:
# del data
gc.collect()

0

In [21]:
train.head()

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-11-16,-3090531316587891587,635554,3.0,1510829829,69.0,99.0,121443283,31236343,0,0.214286,30.0,106.0,18.0,0.000000,0,11931.000000,7800.000002,266.0,"0,158.74303937500002,0,40.03264,0","0,101.23008015625,0,26.500480000000003,0","0,4.00896,0,1.12768,0",29.323308,0.0,1.0,0.0,5.0,22.0,26.0,0.928571,0.071429,0.964286,0.035714,0.785714,28.0,4.714286,4.642857,4.714286,0.023148,0.023324,0.100000,0.111111,0.004630,0.005831,0.023221,0.026537,0.192982,0.215686,0.005435,0.007120,0.025980,0.029265,0.177419,0.211538,0.005014,0.006849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,6.0,15.0,12.0,5.0,1.0,1.0,2.0,8.0,3.0,2.0,4.0,6.0,1.0,0.0,0.0,13.0,70.0,131.0,509.500000,3088.000000,383.500000,2088.000002,104.5,638.5,0.122079,0.131769,9.0,1.0,0.105263,0.003759,12.0,101,72.0,0.583333,0.686038,0.863938,7.831747e+06,46482.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-11-16
2017-11-16,-333165808818623402,1380064,5.0,1510823367,53.5,87.5,114008683,22553459,0,0.327381,385.0,815.0,194.0,0.000000,0,169697.000000,156928.999954,2545.0,"0,3769.3802496,169.921408,3190.8208640000003,0","0,3400.6136576000004,166.19712,3163.6205568000...","0,63.825664,4.128256,44.9008128,0",61.661690,7.0,5.0,13.0,35.0,199.0,205.0,0.984556,0.007722,0.903475,0.046332,0.768340,259.0,4.598455,4.918919,4.598455,0.058335,0.072193,0.133574,0.154867,0.008084,0.011698,0.058040,0.076093,0.141225,0.161820,0.008239,0.012392,0.058262,0.077123,0.137811,0.156780,0.008049,0.012128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193.0,95.0,41.0,18.0,8.0,7.0,85.0,93.0,44.0,28.0,0.0,0.0,0.0,0.0,79.0,612.0,1114.0,4565.500000,40413.500000,4141.000000,38815.500000,924.0,7804.0,0.105606,0.085764,110.0,6.0,0.101768,0.002358,38.0,140,73.0,0.665975,0.956493,0.984987,1.112637e+07,35928.300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-11-16
2017-11-16,-3381759551102345373,1159402,2.5,1510805096,16.5,27.5,121592163,29859320,1,0.338046,263.0,482.0,174.0,0.500000,2,52555.500000,33936.000000,1568.0,"0

### feature engineering

In [61]:
to_drop = [
    "uuid", "poi_id", "wm_poi_id", "reorder", "dt", "order_time", "user_id", "wk", 'mo', 'city_id', "is_discount_fee",
    "is_donation", "shipping_fee", "id", "total", "original_price", "longitude", "latitude", "timescope_order_cnt",
    "timescope_total_price", "timescope_original_price","user_poi_order_decay"
]

fea_drop =["tag_id"]

In [62]:
label = 'reorder'

In [63]:
basic_features = list((set(train.columns)) - set(to_drop) - set(fea_drop))
print basic_features,len(basic_features)

['area_30day', 'pv_ctr_1day', 'neg_delivery_comment_rate', 'order_cnt_12hr', 'op_time_30day', 'click_tag_pref_15day', 'poi_reorder_counts', 'user_reorder_count', 'food_comment_cnt', 'pv_cvr_7day', 'pos_delivery_comment_rate', 'service_fee_rate_7day', 'comment_3star', 'order_tag_pref_15day', 'comment_4star', 'pic_comment_cnt', 'comment_5star_rate', 'month_original_price', 'order_cnt_14hr', 'pv_ctr_7day', 'pos_comment_rate', 'order_cnt_10hr', 'order_cnt_20hr', 'avg_price_month', 'order_cnt_21hr', 'order_cnt_17hr', 'poi_reorder_rate', 'pv_cxr_1day', 'comment_uv', 'month_order_cnt', 'order_cnt_22hr', 'order_cnt_6hr', 'submit_tag_pref_7day', 'order_cnt_23hr', 'pv_cxr_15day', 'uv_cvr_1day', 'original_price_1day', 'comment_1star', 'order_cnt_19hr', 'order_tag_pref_30day', 'dp_score', 'dp_avg_price', 'order_cnt_1day', 'original_price_7day', 'order_cnt_2hr', 'poi_one_bought', 'total_price_7day', 'uv_cvr_7day', 'neg_comment_rate', 'food_comment_rate', 'total_price_1day', 'user_poi_click_decay', 

#### basic feature 

In [55]:
for t in [train,test]:
    t.loc[:,'dow'] = t.dt.dt.dayofweek
    t.loc[:,'dom'] = t.dt.dt.day

# train.loc[:,'dow'] = train.dt.dayofweek

# df_2017.loc[:,'weekday'] = df_2017.date.dt.dayofweek

# df_2017.loc[:,'day'] = df_2017.date.dt.day

# df_2017.loc[:,'week'] = df_2017.date.dt.week

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


KeyboardInterrupt: 

In [94]:
test.head(100)

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt
1564,02425FA9929EA23215C5473DA1E5ED241271BC67424E84...,3245844,3.0,1513138533,31.03,39.80,125115624,46611092,0,0.135266,56.0,347.0,38.0,0.666667,2,36506.769789,28076.269741,818.0,"0,271.200943078125,52.064910956249996,208.3684...","0,209.0234612986328,42.428170156250005,165.817...","0,5.1208704,1.2460032,5.231872,0",34.323068,3.0,0.0,11.0,13.0,209.0,205.0,0.991525,0.004237,0.940678,0.012712,0.885593,236.0,4.800848,4.949152,4.800848,0.029561,0.037691,0.044444,0.047619,0.001689,0.002356,0.028380,0.032053,0.129825,0.137931,0.003818,0.004620,0.034554,0.039438,0.136182,0.150250,0.004769,0.006026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,14.0,21.0,17.0,6.0,7.0,8.0,12.0,25.0,25.0,16.0,7.0,1.0,0.0,0.0,0.0,20.0,166.0,385.0,895.879993,7517.519972,696.239994,5809.719949,189.70,1614.60,0.108378,0.106890,-53.0,15.0,0.288509,0.018337,0.0,105,75.0,0.556169,0.791322,1.283434,2.019600e+07,44661.200000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2017-12-13
1565,03E9C391A796119C66219A277639975F3C93B17D90F7D6...,2123592,4.0,1513159407,25.00,25.00,109168688,21502096,1,0.435185,47.0,62.0,20.0,0.333333,1,6346.790016,6181.790015,238.0,"0,117.16784723535156,6.389760000000001,37.2461...","0,114.54640692285156,6.389760000000001,37.2461...","0,4.747008,0.32768,1.86368,2.88",25.973908,3.0,1.0,2.0,4.0,50.0,44.0,0.966667,0.016667,0.900000,0.066667,0.833333,60.0,4.616667,4.883333,4.616667,0.050704,0.027237,0.214286,0.294118,0.016901,0.019455,0.045530,0.043860,0.271739,0.307692,0.013881,0.015949,0.043651,0.043914,0.205714,0.250000,0.009524,0.011941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,5.0,17.0,5.0,2.0,4.0,2.0,0.0,3.0,1.0,3.0,4.0,2.0,0.0,0.0,7.0,54.0,104.0,178.500000,1411.840006,178.500000,1394.840005,137.70,1172.94,0.186915,0.134662,4.0,2.0,0.252101,0.008403,0.0,95,0.0,0.765423,0.999365,1.228895,2.061689e+07,44006.233333,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2017-12-13
1566,03FD43B4294705F3B24315DA4BC3C4C8024669FCEACF83...,3497786,2.0,1513133789,20.80,40.80,126627747,45697513,0,0.21

#### cat fea

In [143]:
train.head()

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt,dow,dom
0,51D0C90792E56D941FAB07A3E65AADEAABF14C5801D8BE...,2466399,0.0,1507996835,21.0,21.0,111663873,40817629,1,0.333333,179.0,353.0,76.0,0.500000,1,27588.500000,25709.720016,853.0,"0,40.878080000000004,45.524992,77.346560000000...","0,38.767820625,43.105649,72.8742787727539,415....","0,1.409024,1.676288,2.8140032,14.064076800000002",30.140352,1.0,1.0,2.0,10.0,122.0,80.0,0.985294,0.000000,0.970588,0.014706,0.897059,136.0,4.845588,4.941176,4.845588,0.016722,0.017555,0.177778,0.189189,0.003822,0.004551,0.021444,0.023798,0.214559,0.209524,0.004784,0.005236,0.020832,0.024494,0.186466,0.181481,0.003944,0.004529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,12.0,13.0,10.0,13.0,7.0,3.0,11.0,18.0,10.0,15.0,19.0,24.0,44.0,22.0,34.0,224.0,457.0,1120.500000,7161.500000,1045.740000,6706.340004,33.0,203.0,0.006233,0.005970,-9.0,2.0,0.159437,0.002345,0.0,11,0.0,0.734899,0.941210,1.193170,2.207733e+07,51185.033333,3.9492,0.0000,0.0000,0.1000,0.1000,0.1014,0.0959,0.5,0.6667,0.0,0.0,0.5,0.6667,0.6667,0.6667,2017-10-15,6,15
1,95C7943ACF7E4271717C533C5FD6439EB681AF7CF2C242...,568227,7.0,1507996836,84.0,94.0,113676704,22946188,0,0.238227,172.0,505.0,138.0,0.333333,1,57845.000000,49436.000000,838.0,"0,342.175744,0,489.256448,1365.5295232000003","0,302.403584,0,437.90208000000007,1204.004992","0,4.2713088,0,6.8405759999999995,19.3310208",58.992840,5.0,2.0,7.0,14.0,67.0,80.0,0.968421,0.021053,0.852632,0.073684,0.705263,95.0,4.431579,4.821053,4.431579,0.057425,0.078660,0.052632,0.059361,0.003140,0.004893,0.060033,0.077626,0.072217,0.082007,0.004379,0.006445,0.057183,0.074129,0.070944,0.080717,0.004078,0.006022,11.0,14.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,16.0,5.0,1.0,0.0,2.0,8.0,18.0,9.0,9.0,11.0,15.0,21.0,26.0,32.0,178.0,318.0,2203.500000,12961.000000,1954.500000,11643.000000,538.5,2777.5,0.065723,0.078722,38.0,3.0,0.113365,0.003580,58.0,9,74.0,0.745203,0.908976,1.056947,1.343935e+07,61006.500000,7.1721,2.4425,2.4425,0.2727,0.2143,0.1111,0.1429,0.0,0.0000,0.0,0.1,0.0,0.0000,0.0000,0.0833,2017-10-15,6,15
2,2D96F9D2D2530DC8F68B4D1B21325DC6DEDDF771B1BEC5...,3322412,4.0,1507996840,17.5,29.0,115851712,2

In [118]:
data[cat_features_one_hot].dtypes

tag_id    int64
dow       int64
dom       int64
dtype: object

In [135]:
np.unique(data['tag_id'])

array([   0,   94,   95,   96,   97,   98,   99,  100,  101,  102,  103,
        104,  105,  106,  107,  108,  109,  110,  111,  112,  113,  114,
        117,  118,  119,  120,  121,  122,  123,  124,  126,  127,  128,
        129,  130,  131,  132,  133,  134,  135,  136,  137,  138,  139,
        140,  141,  142,  143,  144,  145,  146,  147,  148,  149,  150,
        151,  152,  153,  154,  155,  156,  157,  158,  159,  160,  161,
        162,  163,  164,  165,  166,  167,  168,  169,  170,  171,  173,
        174,  175,  176,  177,  178,  179,  180,  181,  182, 2001, 2002,
       2003, 2004, 2005, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008,
       3009, 3010, 3011, 3012, 3013, 3014, 4000, 4001, 4002, 4003, 4004,
       4005, 4006, 4007, 4008, 4009, 4010, 4011, 4012, 4013, 4014, 4015,
       4016, 4017, 4018, 4019, 4020, 4022, 4023, 4024, 4025])

In [139]:
np.unique(test['tag_id'])

array([   0,   94,   95,   96,   97,   98,   99,  100,  101,  102,  103,
        104,  105,  106,  107,  108,  109,  110,  111,  112,  113,  114,
        117,  118,  119,  120,  121,  122,  123,  124,  126,  127,  128,
        129,  130,  131,  132,  133,  134,  135,  136,  137,  138,  139,
        140,  141,  142,  143,  144,  145,  146,  147,  148,  149,  150,
        151,  152,  153,  154,  155,  156,  157,  158,  159,  160,  161,
        162,  163,  164,  165,  166,  167,  168,  169,  170,  171,  173,
        174,  175,  176,  177,  178,  179,  180,  182, 2001, 2002, 2003,
       2004, 2005, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009,
       3010, 3011, 3012, 3013, 3014, 4000, 4001, 4002, 4003, 4004, 4005,
       4006, 4007, 4008, 4009, 4010, 4011, 4012, 4013, 4014, 4015, 4016,
       4017, 4018, 4019, 4020, 4022, 4023, 4024, 4025])

In [147]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

cat_features_one_hot = ['tag_id','dow','dom']

for c in cat_features_one_hot:
    le = LabelEncoder()
    le.fit(pd.concat([train[c],test[c]],axis=0))
    train[c] = le.transform(train[c])
#     test[c] = le.transform(test[c])

enc = OneHotEncoder()
enc.fit(pd.concat([train[cat_features_one_hot],test[cat_features_one_hot]],axis=0))
X_cat = enc.transform(train[cat_features_one_hot])
X_t_cat = enc.transform(test[cat_features_one_hot])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


#### cat count

In [155]:
train.head()

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt,dow,dom,uuid_count
0,51D0C90792E56D941FAB07A3E65AADEAABF14C5801D8BE...,2466399,0.0,1507996835,21.0,21.0,111663873,40817629,1,0.333333,179.0,353.0,76.0,0.500000,1,27588.500000,25709.720016,853.0,"0,40.878080000000004,45.524992,77.346560000000...","0,38.767820625,43.105649,72.8742787727539,415....","0,1.409024,1.676288,2.8140032,14.064076800000002",30.140352,1.0,1.0,2.0,10.0,122.0,80.0,0.985294,0.000000,0.970588,0.014706,0.897059,136.0,4.845588,4.941176,4.845588,0.016722,0.017555,0.177778,0.189189,0.003822,0.004551,0.021444,0.023798,0.214559,0.209524,0.004784,0.005236,0.020832,0.024494,0.186466,0.181481,0.003944,0.004529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,12.0,13.0,10.0,13.0,7.0,3.0,11.0,18.0,10.0,15.0,19.0,24.0,44.0,22.0,34.0,224.0,457.0,1120.500000,7161.500000,1045.740000,6706.340004,33.0,203.0,0.006233,0.005970,-9.0,2.0,0.159437,0.002345,0.0,11,0.0,0.734899,0.941210,1.193170,2.207733e+07,51185.033333,3.9492,0.0000,0.0000,0.1000,0.1000,0.1014,0.0959,0.5,0.6667,0.0,0.0,0.5,0.6667,0.6667,0.6667,2017-10-15,6,14,6
1,95C7943ACF7E4271717C533C5FD6439EB681AF7CF2C242...,568227,7.0,1507996836,84.0,94.0,113676704,22946188,0,0.238227,172.0,505.0,138.0,0.333333,1,57845.000000,49436.000000,838.0,"0,342.175744,0,489.256448,1365.5295232000003","0,302.403584,0,437.90208000000007,1204.004992","0,4.2713088,0,6.8405759999999995,19.3310208",58.992840,5.0,2.0,7.0,14.0,67.0,80.0,0.968421,0.021053,0.852632,0.073684,0.705263,95.0,4.431579,4.821053,4.431579,0.057425,0.078660,0.052632,0.059361,0.003140,0.004893,0.060033,0.077626,0.072217,0.082007,0.004379,0.006445,0.057183,0.074129,0.070944,0.080717,0.004078,0.006022,11.0,14.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,16.0,5.0,1.0,0.0,2.0,8.0,18.0,9.0,9.0,11.0,15.0,21.0,26.0,32.0,178.0,318.0,2203.500000,12961.000000,1954.500000,11643.000000,538.5,2777.5,0.065723,0.078722,38.0,3.0,0.113365,0.003580,58.0,9,74.0,0.745203,0.908976,1.056947,1.343935e+07,61006.500000,7.1721,2.4425,2.4425,0.2727,0.2143,0.1111,0.1429,0.0,0.0000,0.0,0.1,0.0,0.0000,0.0000,0.0833,2017-10-15,6,14,23
2,2D96F9D2D2530DC8F68B4D1B21325DC6DEDDF771B1BEC5...,3322412,4.0,1507996840,17.5,

In [156]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

cat_features = ['uuid','wm_poi_id','tag_id']
cat_count_features = []
for c in cat_features:
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [65]:
len(features)

101

In [ ]:
user* geohash

#### user part 

In [ ]:
user tag_id

In [ ]:
购买频率，15天下单量，15天 reorder 量,未来什么都不买的概率。

In [22]:
data.head()

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-19,-2468829209915391773,267654,5.0,1508405889,153.00,193.0,116460854,39950249,0,0.291059,1058.0,2387.0,762.0,0.200000,1,620158.900780,386613.301579,7406.0,"0,19798.035546683594,2124.08283425,11411.16169...","0,11457.424141970216,1397.784979403125,7250.40...","0,275.9084032,25.6947712,130.8906496,8.5080064...",52.202714,49.0,31.0,50.0,99.0,642.0,700.0,0.954076,0.020666,0.850746,0.091848,0.737084,871.0,4.439724,4.818599,4.439724,0.051241,0.069676,0.151042,0.174584,0.007807,0.012311,0.058617,0.079230,0.174720,0.202407,0.010260,0.016076,0.058786,0.081452,0.157253,0.183576,0.009252,0.014969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,268.0,532.0,217.0,87.0,70.0,43.0,72.0,169.0,175.0,91.0,59.0,35.0,9.0,0.0,0.0,302.0,1827.0,3454.0,23681.400055,143045.000301,15002.300070,89053.300270,5360.0,34050.0,0.108546,0.109902,200.0,29.0,0.117607,0.003916,81.0,131,85.0,0.529240,0.596067,1.206247,18010555.76,42636.666667,0.0,0.0,0.0,0.00,0.00,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-19
2017-10-19,-5321962128904941823,2004463,4.0,1508412861,26.00,29.0,102693461,25042763,0,0.276973,172.0,428.0,108.0,0.000000,0,98315.000000,75350.080008,1209.0,"40.32,2030.7116544,252.16558080000001,2550.755...","24.96,1575.0197238844728,185.1088388272461,198...","0.64,22.786406400000004,3.6780544,27.3882112,5...",62.324301,3.0,1.0,7.0,26.0,310.0,270.0,0.994236,0.002882,0.968300,0.011527,0.893372,347.0,4.841498,4.951009,4.841498,0.023371,0.030716,0.072000,0.070423,0.001753,0.002270,0.028804,0.037537,0.093250,0.095238,0.002705,0.003604,0.030646,0.040947,0.092725,0.095413,0.002850,0.003920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,13.0,46.0,25.0,19.0,9.0,11.0,15.0,35.0,43.0,25.0,18.0,29.0,0.0,0.0,0.0,43.0,295.0,578.0,3098.000000,23730.000000,2240.880001,18440.880005,678.0,4945.0,0.081464,0.056375,12.0,10.0,0.287014,0.008271,90.0,164,75.0,0.609547,0.746001,1.100069,13325026.83,42650.733333,0.0,0.0,0.0,0.00,0.00,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-19
2017-10-19,008BCA6D9838

In [23]:
pd.to_datetime('20171001') - pd.Timedelta('1 days')

Timestamp('2017-09-30 00:00:00')

In [24]:
def get_past(end, periods):
    return pd.date_range(end=end - pd.Timedelta('1 days'),periods=periods).intersection(np.unique(data['dt']))

get_past(pd.to_datetime('20171031'),20)

DatetimeIndex(['2017-10-15', '2017-10-16', '2017-10-17', '2017-10-18',
               '2017-10-19', '2017-10-20', '2017-10-21', '2017-10-22',
               '2017-10-23', '2017-10-24', '2017-10-25', '2017-10-26',
               '2017-10-27', '2017-10-28', '2017-10-29', '2017-10-30'],
              dtype='datetime64[ns]', freq='D')

In [25]:
from tqdm import *
for i in tqdm(range(100)):
    pass

100%|██████████| 100/100 [00:00<00:00, 632625.04it/s]


In [26]:
pd.date_range('20171116',periods=10)

DatetimeIndex(['2017-11-16', '2017-11-17', '2017-11-18', '2017-11-19',
               '2017-11-20', '2017-11-21', '2017-11-22', '2017-11-23',
               '2017-11-24', '2017-11-25'],
              dtype='datetime64[ns]', freq='D')

In [27]:
data.head()

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-19,-2468829209915391773,267654,5.0,1508405889,153.00,193.0,116460854,39950249,0,0.291059,1058.0,2387.0,762.0,0.200000,1,620158.900780,386613.301579,7406.0,"0,19798.035546683594,2124.08283425,11411.16169...","0,11457.424141970216,1397.784979403125,7250.40...","0,275.9084032,25.6947712,130.8906496,8.5080064...",52.202714,49.0,31.0,50.0,99.0,642.0,700.0,0.954076,0.020666,0.850746,0.091848,0.737084,871.0,4.439724,4.818599,4.439724,0.051241,0.069676,0.151042,0.174584,0.007807,0.012311,0.058617,0.079230,0.174720,0.202407,0.010260,0.016076,0.058786,0.081452,0.157253,0.183576,0.009252,0.014969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,268.0,532.0,217.0,87.0,70.0,43.0,72.0,169.0,175.0,91.0,59.0,35.0,9.0,0.0,0.0,302.0,1827.0,3454.0,23681.400055,143045.000301,15002.300070,89053.300270,5360.0,34050.0,0.108546,0.109902,200.0,29.0,0.117607,0.003916,81.0,131,85.0,0.529240,0.596067,1.206247,18010555.76,42636.666667,0.0,0.0,0.0,0.00,0.00,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-19
2017-10-19,-5321962128904941823,2004463,4.0,1508412861,26.00,29.0,102693461,25042763,0,0.276973,172.0,428.0,108.0,0.000000,0,98315.000000,75350.080008,1209.0,"40.32,2030.7116544,252.16558080000001,2550.755...","24.96,1575.0197238844728,185.1088388272461,198...","0.64,22.786406400000004,3.6780544,27.3882112,5...",62.324301,3.0,1.0,7.0,26.0,310.0,270.0,0.994236,0.002882,0.968300,0.011527,0.893372,347.0,4.841498,4.951009,4.841498,0.023371,0.030716,0.072000,0.070423,0.001753,0.002270,0.028804,0.037537,0.093250,0.095238,0.002705,0.003604,0.030646,0.040947,0.092725,0.095413,0.002850,0.003920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,13.0,46.0,25.0,19.0,9.0,11.0,15.0,35.0,43.0,25.0,18.0,29.0,0.0,0.0,0.0,43.0,295.0,578.0,3098.000000,23730.000000,2240.880001,18440.880005,678.0,4945.0,0.081464,0.056375,12.0,10.0,0.287014,0.008271,90.0,164,75.0,0.609547,0.746001,1.100069,13325026.83,42650.733333,0.0,0.0,0.0,0.00,0.00,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-19
2017-10-19,008BCA6D9838

In [28]:
uuid_fea_list = []
for tgt_date in tqdm(pd.date_range('20171116',periods=10).append(pd.date_range('20171210',periods=5))):
    print tgt_date
#     tgt_date = '20171031'
    data_4_feas = data.loc[get_past(pd.to_datetime(tgt_date),15),:]
    uuid_fea = pd.DataFrame({"uuid":data_4_feas.uuid.unique()})
    data_4_feas.loc[:,'order_id'] = 0
    gby_cols = ["uuid"]
#     ['mean', 'median', 'max', 'min', 'std','skew']
    uuid_reorder_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['reorder'],
                                   tgt_stats=["mean", "median", "std",'skew'],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="_")
    uuid_fea = pd.merge(uuid_fea,uuid_reorder_agg,on=gby_cols,how='left')

    gby_cols = ["uuid"]
    uuid_price_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['total'],
    #                                tgt_stats=["mean", "median", "std"],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="price")
    uuid_fea = pd.merge(uuid_fea,uuid_price_agg,on=gby_cols,how='left')

    gby_cols = ["uuid"]
    uuid_reorder_price_agg = get_stats_target(data_4_feas[data_4_feas.reorder>0],
                                   gby_cols,
                                   ['total'],
    #                                tgt_stats=["mean", "median", "std"],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="reorder_price")

    uuid_fea = pd.merge(uuid_fea,uuid_reorder_price_agg,on=gby_cols,how='left')
    uuid_fea['dt'] = pd.to_datetime(tgt_date)
    uuid_fea_list.append(uuid_fea)

  0%|          | 0/15 [00:00<?, ?it/s]

2017-11-16 00:00:00


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


prefix is _,group column is uuid,agg total count is 202585; after filter , agg total count is 202585, filter ratio is 4.93619938258e-06 %
prefix is price,group column is uuid,agg total count is 202585; after filter , agg total count is 202585, filter ratio is 4.93619938258e-06 %


  7%|▋         | 1/15 [01:48<25:18, 108.46s/it]

...

prefix is reorder_price,group column is uuid,agg total count is 80717; after filter , agg total count is 80717, filter ratio is 1.23889623671e-05 %
2017-12-13 00:00:00
prefix is _,group column is uuid,agg total count is 214917; after filter , agg total count is 214917, filter ratio is 4.65295884045e-06 %
prefix is price,group column is uuid,agg total count is 214917; after filter , agg total count is 214917, filter ratio is 4.65295884045e-06 %


 93%|█████████▎| 14/15 [25:50<01:50, 110.78s/it]

prefix is reorder_price,group column is uuid,agg total count is 81023; after filter , agg total count is 81023, filter ratio is 1.2342172917e-05 %
2017-12-14 00:00:00
prefix is _,group column is uuid,agg total count is 215594; after filter , agg total count is 215594, filter ratio is 4.63834779474e-06 %
prefix is price,group column is uuid,agg total count is 215594; after filter , agg total count is 215594, filter ratio is 4.63834779474e-06 %


100%|██████████| 15/15 [27:43<00:00, 110.90s/it]

prefix is reorder_price,group column is uuid,agg total count is 81293; after filter , agg total count is 81293, filter ratio is 1.23011806186e-05 %


In [29]:
uuid_reorder_agg.head(100)

,___reorder__mean_by__uuid,___reorder__median_by__uuid,___reorder__std_by__uuid,___reorder__skew_by__uuid,uuid,___order_id__count_by__uuid
0,0.000000,0.0,NaN,NaN,-1001338100389457672,1
1,0.000000,0.0,NaN,NaN,-1002908462155165269,1
2,0.000000,0.0,0.000000,0.000000,-1002950902334329163,3
3,0.000000,0.0,NaN,NaN,-1003223898460162205,1
4,0.000000,0.0,NaN,NaN,-1003837415485152900,1
5,0.000000,0.0,0.000000,NaN,-1004502039539197946,2
6,0.000000,0.0,NaN,NaN,-1006632276068093342,1
7,0.000000,0.0,0.000000,0.000000,-1007488885529299052,3
8,0.250000,0.0,0.500000,2.000000,-101124832082636328,4
9,0.000000,0.0,0.000000,0.000000,-1015857463153752563,3


In [41]:
uuid_fea2_list = []
def uniq(x):
    return x.nunique()

for tgt_date in tqdm(pd.date_range('20171116',periods=10).append(pd.date_range('20171210',periods=5))):
    print tgt_date
#     tgt_date = '20171031'
    def days_since_last_order(x):
#         print x,"????"
#         if len(x)>0:
        return (tgt_date - x.sort_values().iloc[-1])/ np.timedelta64(1, 'D')
    
#     def days_since_last_reorder(x):
#         if len(x[data_4_feas.ix[x.index,'reorder']==1])>0:
#             return (tgt_date - x[data_4_feas.ix[x.index,'reorder']==1].sort_values().iloc[-1])/ np.timedelta64(1, 'D')
#         else :
#             return 999
        
    data_4_feas = data.loc[get_past(pd.to_datetime(tgt_date),15),:]
    data_4_feas.index=range(data_4_feas.shape[0])
    uuid_fea = pd.DataFrame({"uuid":data_4_feas.uuid.unique()})
    data_4_feas.loc[:,'order_id'] = 0
    gby_cols = ["uuid"]
#     ['mean', 'median', 'max', 'min', 'std','skew']

    uuid_reorder_agg = ka_create_groupby_features(data_4_feas,
                                   gby_cols,
                                   {
#                                        "wm_poi_id":["count",uniq],
                                    "dt":[
                                        days_since_last_order,
#                                           days_since_last_reorder
                                    ]
                                   })
    uuid_fea = pd.merge(uuid_fea,uuid_reorder_agg,on=gby_cols,how='left')
    uuid_fea['dt'] = pd.to_datetime(tgt_date)
    uuid_fea2_list.append(uuid_fea)

  0%|          | 0/15 [00:00<?, ?it/s]

2017-11-16 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 53.594648838 s 



  7%|▋         | 1/15 [00:58<13:38, 58.45s/it]

2017-11-17 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 59.5185759068 s 



 13%|█▎        | 2/15 [02:02<13:13, 61.03s/it]

...

 87%|████████▋ | 13/15 [13:11<02:01, 60.91s/it]

2017-12-13 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 62.0850629807 s 



 93%|█████████▎| 14/15 [14:17<01:01, 61.28s/it]

2017-12-14 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 56.9478509426 s 



100%|██████████| 15/15 [15:19<00:00, 61.29s/it]


In [34]:
uuid_fea.head()

,uuid,uuid_G_dt_days_since_last_order
0,038F2DC1B7BB4A388DE25DD941B8BE315281C66AD8CBD0...,5.0
1,0398EECCF568907EDE8D772AF8E960D5459A7032897F02...,1.0
2,079A19AD3DF0F19DA31953D2D6AA53AA9A646CF7C0E5F7...,3.0
3,0B23FC501591984588EA5C9F3EA4AAA1748037F7B62111...,2.0
4,0B37CB790CD857054323AF83521324EB36C8EC339CBB70...,12.0


In [443]:
t = data_4_feas.groupby(['uuid']).apply(lambda x: x[x.reorder > 0].dt.sort_values().diff().mean()).reset_index()
t.columns=[
    'uuid','reorder_gap_mean'
]
uuid_fea = uuid_fea.merge(t,on=['uuid'],how='inner')
uuid_fea.reorder_gap_mean.fillna(999,inplace=True)
uuid_fea['reorder_gap_mean']=(uuid_fea.reorder_gap_mean/ np.timedelta64(1, 'D')).astype(int)

t = train.groupby(['uuid']).apply(lambda x: x.dt.sort_values().diff().mean()).reset_index()
t.columns=[
    'uuid','order_gap_mean'
]
uuid_fea = uuid_fea.merge(t,on=['uuid'],how='inner')
uuid_fea.order_gap_mean.fillna(999,inplace=True)s
uuid_fea['order_gap_mean']=(uuid_fea.order_gap_mean/ np.timedelta64(1, 'D')).astype(int)
uuid_fea_list.append(uuid_fea)

SyntaxError: invalid syntax (<ipython-input-443-052e433074f4>, line 14)

In [43]:
uuid_fea = pd.concat(uuid_fea_list)
uuid_fea2 = pd.concat(uuid_fea2_list)

In [36]:
uuid_fea.head()

,uuid,___reorder__mean_by__uuid,___reorder__median_by__uuid,___reorder__std_by__uuid,___reorder__skew_by__uuid,___order_id__count_by__uuid,price__total__mean_by__uuid,price__total__median_by__uuid,price__total__max_by__uuid,price__total__min_by__uuid,price__total__std_by__uuid,price__total__skew_by__uuid,price__order_id__count_by__uuid,reorder_price__total__mean_by__uuid,reorder_price__total__median_by__uuid,reorder_price__total__max_by__uuid,reorder_price__total__min_by__uuid,reorder_price__total__std_by__uuid,reorder_price__total__skew_by__uuid,reorder_price__order_id__count_by__uuid,dt
0,0884617FA79D122ACB81109736EA73E52EDA3FFD723031...,0.900000,1.0,0.316228,-3.162278,10,17.810000,17.150,25.3,12.0,4.455072,0.319639,10,17.866667,17.0,25.3,12.0,4.721493,0.26463,9,2017-11-16
1,09FE3DD5DC5FCDE7EC87D9BC9C51AC27410B7014F7F2B6...,0.333333,0.0,0.516398,0.968246,6,26.348333,12.895,61.0,6.5,24.001175,0.969995,6,36.400000,36.4,61.0,11.8,34.789654,NaN,2,2017-11-16
2,143D85ADB0B0C38871D66449991336442423A116E4D241...,0.666667,1.0,0.516398,-0.968246,6,13.000000,11.000,23.0,11.0,4.898979,2.449490,6,11.000000,11.0,11.0,11.0,0.000000,0.00000,4,2017-11-16
3,153EDA177FD794A9085A138B6A2B294AF51BB58387EB44...,0.111111,0.0,0.333333,3.000000,9,23.955556,23.500,31.5,15.3,4.648417,-0.108496,9,23.500000,23.5,23.5,23.5,NaN,NaN,1,2017-11-16
4,15a6071b4340-0d626cee2d483-5c206d23-38400-15a6...,0.500000,0.5,0.707107,NaN,2,28.500000,28.500,29.5,27.5,1.414214,NaN,2,29.500000,29.5,29.5,29.5,NaN,NaN,1,2017-11-16


In [37]:
uuid_fea2.head()

,uuid,uuid_G_dt_days_since_last_order
0,-7171747965525239881,14.0
1,03B0A314A60BE0F419B4EA7491E9CF969C200974751B68...,15.0
2,04BA099C6A95D66D51EEA7CEE23A20CD177B868459B6CC...,1.0
3,0884617FA79D122ACB81109736EA73E52EDA3FFD723031...,1.0
4,09FE3DD5DC5FCDE7EC87D9BC9C51AC27410B7014F7F2B6...,9.0


In [221]:
t = train.groupby(['uuid']).apply(lambda x: x[x.reorder > 0].dt.sort_values().diff().mean()).reset_index()
t.columns=[
    'uuid','reorder_gap_mean'
]
uuid_fea = uuid_fea.merge(t,on=['uuid'],how='inner')
uuid_fea.reorder_gap_mean.fillna(999,inplace=True)
uuid_fea['reorder_gap_mean']=(uuid_fea.reorder_gap_mean/ np.timedelta64(1, 'D')).astype(int)

In [222]:
uuid_fea.head()

,uuid,reorder_gap_mean
0,-2468829209915391773,0
1,-5321962128904941823,0
2,008BCA6D9838489221D4E3EA42E4CB6B24BD87C1E09818...,10
3,015112C211090CF5C2E2141D170E89976E9B540C3E4BD7...,0
4,0273CCDDF855F987B1BAFFF3E2C2DF7D719BD0F4CFD979...,2


In [223]:
t = train.groupby(['uuid']).apply(lambda x: x.dt.sort_values().diff().mean()).reset_index()
t.columns=[
    'uuid','order_gap_mean'
]
uuid_fea = uuid_fea.merge(t,on=['uuid'],how='inner')
uuid_fea.order_gap_mean.fillna(999,inplace=True)
uuid_fea['order_gap_mean']=(uuid_fea.order_gap_mean/ np.timedelta64(1, 'D')).astype(int)

In [224]:
train['order_id'] = train['uuid']

prefix is _,group column is uuid,agg total count is 280944; after filter , agg total count is 280944, filter ratio is 3.55942808561e-06 %


prefix is price,group column is uuid,agg total count is 280944; after filter , agg total count is 280944, filter ratio is 3.55942808561e-06 %


prefix is reorder_price,group column is uuid,agg total count is 102571; after filter , agg total count is 102571, filter ratio is 9.74934339926e-06 %


In [228]:
uuid_fea.head()

,uuid,reorder_gap_mean,order_gap_mean,___reorder__mean_by__uuid,___reorder__median_by__uuid,___reorder__max_by__uuid,___reorder__min_by__uuid,___reorder__std_by__uuid,___order_id__count_by__uuid,price__original_price__mean_by__uuid,price__original_price__median_by__uuid,price__original_price__max_by__uuid,price__original_price__min_by__uuid,price__original_price__std_by__uuid,price__order_id__count_by__uuid,reorder_price__original_price__mean_by__uuid,reorder_price__original_price__median_by__uuid,reorder_price__original_price__max_by__uuid,reorder_price__original_price__min_by__uuid,reorder_price__original_price__std_by__uuid,reorder_price__order_id__count_by__uuid
0,-2468829209915391773,0,3,0.111111,0.0,1,0,0.333333,9,162.000000,166.50,262.0,92.0,54.143905,9,92.000,92.00,92.0,92.0,NaN,1
1,-5321962128904941823,0,2,0.111111,0.0,1,0,0.333333,9,59.500000,52.00,90.0,29.0,26.802519,9,87.000,87.00,87.0,87.0,NaN,1
2,008BCA6D9838489221D4E3EA42E4CB6B24BD87C1E09818...,10,1,0.187500,0.0,1,0,0.403113,16,49.743750,38.10,160.5,18.0,42.123659,16,30.100,25.00,40.3,25.0,8.833459,3
3,0273CCDDF855F987B1BAFFF3E2C2DF7D719BD0F4CFD979...,2,5,0.666667,1.0,1,0,0.516398,6,58.833333,46.25,119.0,32.0,33.681845,6,67.625,59.25,119.0,33.0,39.045646,4
4,02AAADEB283125D3138AF07AD2E2773C5E9B9601185CC1...,0,8,0.500000,0.5,1,0,0.577350,4,36.450000,26.75,82.3,10.0,31.602373,4,17.500,17.50,25.0,10.0,10.606602,2


In [ ]:
#Time between orders

In [ ]:
#how long user buy a new poi

In [ ]:
#realtime feature
#last rebuy tag_id

#### poi part

In [ ]:
print "平均被复购的时间"

In [96]:
poi_fea_list = []
for tgt_date in tqdm(pd.date_range('20171116',periods=10).append(pd.date_range('20171210',periods=5))):
    print tgt_date
#     tgt_date = '20171031'
    data_4_feas = data.loc[get_past(pd.to_datetime(tgt_date),15),:]
    poi_fea = pd.DataFrame({"wm_poi_id":data_4_feas.wm_poi_id.unique()})
    data_4_feas.loc[:,'order_id'] = 0
    gby_cols = ["wm_poi_id"]
#     ['mean', 'median', 'max', 'min', 'std','skew']
    uuid_reorder_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['reorder'],
                                   tgt_stats=["mean", "median", "std",'skew'],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="_")
    poi_fea = pd.merge(poi_fea,uuid_reorder_agg,on=gby_cols,how='left')

    gby_cols = ["wm_poi_id"]
    uuid_price_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['total'],
    #                                tgt_stats=["mean", "median", "std"],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="price")
    poi_fea = pd.merge(poi_fea,uuid_price_agg,on=gby_cols,how='left')

    gby_cols = ["wm_poi_id"]
    uuid_reorder_price_agg = get_stats_target(data_4_feas[data_4_feas.reorder>0],
                                   gby_cols,
                                   ['total'],
    #                                tgt_stats=["mean", "median", "std"],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="reorder_price")

    poi_fea = pd.merge(poi_fea,uuid_reorder_price_agg,on=gby_cols,how='left')
    poi_fea['dt'] = pd.to_datetime(tgt_date)
    poi_fea_list.append(poi_fea)

  0%|          | 0/15 [00:00<?, ?it/s]

2017-11-16 00:00:00
prefix is _,group column is wm_poi_id,agg total count is 292829; after filter , agg total count is 292829, filter ratio is 3.41496220191e-06 %
prefix is price,group column is wm_poi_id,agg total count is 292829; after filter , agg total count is 292829, filter ratio is 3.41496220191e-06 %


  7%|▋         | 1/15 [02:17<32:06, 137.64s/it]

prefix is reorder_price,group column is wm_poi_id,agg total count is 97434; after filter , agg total count is 97434, filter ratio is 1.02633566978e-05 %
2017-11-17 00:00:00
prefix is _,group column is wm_poi_id,agg total count is 294222; after filter , agg total count is 294222, filter ratio is 3.39879399069e-06 %
prefix is price,group column is wm_poi_id,agg total count is 294222; after filter , agg total count is 294222, filter ratio is 3.39879399069e-06 %


 13%|█▎        | 2/15 [04:23<28:35, 131.99s/it]

...

prefix is reorder_price,group column is wm_poi_id,agg total count is 103951; after filter , agg total count is 103951, filter ratio is 9.61991614146e-06 %
2017-12-13 00:00:00
prefix is _,group column is wm_poi_id,agg total count is 311425; after filter , agg total count is 311425, filter ratio is 3.21104589585e-06 %
prefix is price,group column is wm_poi_id,agg total count is 311425; after filter , agg total count is 311425, filter ratio is 3.21104589585e-06 %


 93%|█████████▎| 14/15 [31:50<02:16, 136.43s/it]

prefix is reorder_price,group column is wm_poi_id,agg total count is 104323; after filter , agg total count is 104323, filter ratio is 9.58561299225e-06 %
2017-12-14 00:00:00
prefix is _,group column is wm_poi_id,agg total count is 312586; after filter , agg total count is 312586, filter ratio is 3.19911950264e-06 %
prefix is price,group column is wm_poi_id,agg total count is 312586; after filter , agg total count is 312586, filter ratio is 3.19911950264e-06 %


100%|██████████| 15/15 [34:13<00:00, 136.92s/it]

prefix is reorder_price,group column is wm_poi_id,agg total count is 104696; after filter , agg total count is 104696, filter ratio is 9.55146236548e-06 %


In [97]:
poi_fea = pd.concat(poi_fea_list)

In [98]:
poi_fea.head()

,wm_poi_id,___reorder__mean_by__wm_poi_id,___reorder__median_by__wm_poi_id,___reorder__std_by__wm_poi_id,___reorder__skew_by__wm_poi_id,___order_id__count_by__wm_poi_id,price__total__mean_by__wm_poi_id,price__total__median_by__wm_poi_id,price__total__max_by__wm_poi_id,price__total__min_by__wm_poi_id,price__total__std_by__wm_poi_id,price__total__skew_by__wm_poi_id,price__order_id__count_by__wm_poi_id,reorder_price__total__mean_by__wm_poi_id,reorder_price__total__median_by__wm_poi_id,reorder_price__total__max_by__wm_poi_id,reorder_price__total__min_by__wm_poi_id,reorder_price__total__std_by__wm_poi_id,reorder_price__total__skew_by__wm_poi_id,reorder_price__order_id__count_by__wm_poi_id,dt
0,294039,0.000000,0.0,0.000000,0.000000,4,36.250000,28.50,65.0,23.0,19.551215,1.778462,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-16
1,497258,0.000000,0.0,0.000000,0.000000,6,36.113333,34.75,64.0,10.0,19.383113,0.171525,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-16
2,2296536,0.400000,0.0,0.547723,0.608581,5,20.000000,22.00,29.0,11.0,7.745967,-0.188270,5,27.000000,27.0,29.0,25.0,2.828427,NaN,2.0,2017-11-16
3,2407874,0.857143,1.0,0.377964,-2.645751,7,17.328571,16.00,29.6,12.0,6.107022,1.618359,7,15.283333,14.6,20.3,12.0,3.101236,0.786559,6.0,2017-11-16
4,3204360,0.500000,0.5,0.707107,NaN,2,12.800000,12.80,13.8,11.8,1.414214,NaN,2,11.800000,11.8,11.8,11.8,NaN,NaN,1.0,2017-11-16


#### user*poi

In [ ]:
days_since_last_order_this_item

In [95]:
data_4_feas.count()

uuid                               693019
wm_poi_id                          693019
shipping_fee                       693019
order_time                         693019
total                              693019
original_price                     693019
longitude                          693019
latitude                           693019
reorder                            693019
poi_reorder_rate                   693019
poi_reorder_counts                 693019
poi_one_bought                     693019
poi_two_bought                     693019
user_reorder_rate                  693019
user_reorder_count                 693019
month_original_price               693019
month_total_price                  693019
month_order_cnt                    693019
timescope_original_price           693019
timescope_total_price              693019
timescope_order_cnt                693019
avg_price_month                    693019
comment_1star                      693019
comment_2star                     

In [94]:
data_4_feas[['uuid','wm_poi_id']].drop_duplicates().count()

uuid         532029
wm_poi_id    532029
dtype: int64

In [101]:
poiuuid_fea_list = []
uuidtagid_fea_list=[]
for tgt_date in tqdm(pd.date_range('20171116',periods=10).append(pd.date_range('20171210',periods=5))):
    print tgt_date
#     tgt_date = '20171031'

    data_4_feas = data.loc[get_past(pd.to_datetime(tgt_date),15),:]
    poiuuid_fea = data_4_feas[['uuid','wm_poi_id']].drop_duplicates()
    data_4_feas.loc[:,'order_id'] = 0
    gby_cols = ["uuid","wm_poi_id"]
#     ['mean', 'median', 'max', 'min', 'std','skew']
    uuid_reorder_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['reorder'],
                                   tgt_stats=["mean", "median", "std",'skew'],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="_")
    poiuuid_fea = pd.merge(poiuuid_fea,uuid_reorder_agg,on=gby_cols,how='left')

    gby_cols = ["uuid","wm_poi_id"]
    uuid_price_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['total'],
    #                                tgt_stats=["mean", "median", "std"],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="price")
    poiuuid_fea = pd.merge(poiuuid_fea,uuid_price_agg,on=gby_cols,how='left')
    
    uuidtagid_fea = data_4_feas[['uuid','tag_id']].drop_duplicates()
    
    gby_cols = ["uuid","tag_id"]
#     ['mean', 'median', 'max', 'min', 'std','skew']
    uuid_reorder_agg = get_stats_target(data_4_feas,
                                   gby_cols,
                                   ['reorder'],
                                   tgt_stats=["mean", "median", "std",'skew'],
                                   drop_count=False,
                                   filter_count=1,
                                   prefix="_")
    
    uuidtagid_fea = pd.merge(uuidtagid_fea,uuid_reorder_agg,on=gby_cols,how='left')

    poiuuid_fea['dt'] = pd.to_datetime(tgt_date)
    uuidtagid_fea['dt'] = pd.to_datetime(tgt_date)
    
    uuidtagid_fea_list.append(uuidtagid_fea)
    poiuuid_fea_list.append(poiuuid_fea)

  0%|          | 0/15 [00:00<?, ?it/s]

2017-11-16 00:00:00
prefix is _,group column is uuid#wm_poi_id,agg total count is 490562; after filter , agg total count is 490562, filter ratio is 2.03847827507e-06 %
prefix is price,group column is uuid#wm_poi_id,agg total count is 490562; after filter , agg total count is 490562, filter ratio is 2.03847827507e-06 %
prefix is _,group column is uuid#tag_id,agg total count is 422062; after filter , agg total count is 422062, filter ratio is 2.36932009567e-06 %


  7%|▋         | 1/15 [04:19<1:00:36, 259.78s/it]

2017-11-17 00:00:00
prefix is _,group column is uuid#wm_poi_id,agg total count is 494169; after filter , agg total count is 494169, filter ratio is 2.02359917711e-06 %
prefix is price,group column is uuid#wm_poi_id,agg total count is 494169; after filter , agg total count is 494169, filter ratio is 2.02359917711e-06 %
prefix is _,group column is uuid#tag_id,agg total count is 425074; after filter , agg total count is 425074, filter ratio is 2.35253150294e-06 %


 13%|█▎        | 2/15 [08:34<55:41, 257.03s/it]  

...

 87%|████████▋ | 13/15 [57:33<08:51, 265.68s/it]

2017-12-13 00:00:00
prefix is _,group column is uuid#wm_poi_id,agg total count is 529797; after filter , agg total count is 529797, filter ratio is 1.88751537733e-06 %
prefix is price,group column is uuid#wm_poi_id,agg total count is 529797; after filter , agg total count is 529797, filter ratio is 1.88751537733e-06 %
prefix is _,group column is uuid#tag_id,agg total count is 453033; after filter , agg total count is 453033, filter ratio is 2.20734467371e-06 %


 93%|█████████▎| 14/15 [1:02:12<04:26, 266.64s/it]

2017-12-14 00:00:00
prefix is _,group column is uuid#wm_poi_id,agg total count is 532029; after filter , agg total count is 532029, filter ratio is 1.87959675602e-06 %
prefix is price,group column is uuid#wm_poi_id,agg total count is 532029; after filter , agg total count is 532029, filter ratio is 1.87959675602e-06 %
prefix is _,group column is uuid#tag_id,agg total count is 454785; after filter , agg total count is 454785, filter ratio is 2.1988411647e-06 %


100%|██████████| 15/15 [1:06:53<00:00, 267.55s/it]


In [111]:
poiuuid_fea2_list = []
def uniq(x):
    return x.nunique()

for tgt_date in tqdm(pd.date_range('20171116',periods=10).append(pd.date_range('20171210',periods=5))):
    print tgt_date
#     tgt_date = '20171031'
    def days_since_last_order(x):
#         print x,"????"
#         if len(x)>0:
        return (tgt_date - x.sort_values().iloc[-1])/ np.timedelta64(1, 'D')
#         else:
#             return 999
    
#     def days_since_last_reorder(x):
#         if len(x[data_4_feas.ix[x.index,'reorder']==1])>0:
#             return (tgt_date - x[data_4_feas.ix[x.index,'reorder']==1].sort_values().iloc[-1])/ np.timedelta64(1, 'D')
#         else :
#             return 999
        
    data_4_feas = data.loc[get_past(pd.to_datetime(tgt_date),15),:]
    data_4_feas.index=range(data_4_feas.shape[0])
    poiuuid_fea = data_4_feas[['uuid','wm_poi_id']].drop_duplicates()
    
    data_4_feas.loc[:,'order_id'] = 0
    gby_cols = ["uuid","wm_poi_id"]
#     ['mean', 'median', 'max', 'min', 'std','skew']

    uuid_reorder_agg = ka_create_groupby_features(data_4_feas,
                                   gby_cols,
                                   {
#                                        "wm_poi_id":["count",uniq],
                                    "dt":[
                                        days_since_last_order,
#                                           days_since_last_reorder
                                    ]
                                   })
    poiuuid_fea = pd.merge(poiuuid_fea,uuid_reorder_agg,on=gby_cols,how='left')
    poiuuid_fea['dt'] = pd.to_datetime(tgt_date)
    poiuuid_fea2_list.append(poiuuid_fea)

  0%|          | 0/15 [00:00<?, ?it/s]

2017-11-16 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 138.740966797 s 



  7%|▋         | 1/15 [02:31<35:14, 151.04s/it]

2017-11-17 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 138.739102125 s 



 13%|█▎        | 2/15 [04:54<31:55, 147.34s/it]

...

 87%|████████▋ | 13/15 [29:31<04:32, 136.29s/it]

2017-12-13 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 133.426073074 s 



 93%|█████████▎| 14/15 [31:49<02:16, 136.38s/it]

2017-12-14 00:00:00
create stats features begin ......
[('dt', 'days_since_last_order')]
create stats features end ......
time lapsing 134.13902688 s 



100%|██████████| 15/15 [34:07<00:00, 136.51s/it]


In [112]:
uuidtagid_fea = pd.concat(uuidtagid_fea_list)
poiuuid_fea = pd.concat(poiuuid_fea_list)
poiuuid_fea2 = pd.concat(poiuuid_fea2_list)

In [113]:
poiuuid_fea.head()

,uuid,wm_poi_id,___reorder__mean_by__uuid#wm_poi_id,___reorder__median_by__uuid#wm_poi_id,___reorder__std_by__uuid#wm_poi_id,___reorder__skew_by__uuid#wm_poi_id,___order_id__count_by__uuid#wm_poi_id,price__total__mean_by__uuid#wm_poi_id,price__total__median_by__uuid#wm_poi_id,price__total__max_by__uuid#wm_poi_id,price__total__min_by__uuid#wm_poi_id,price__total__std_by__uuid#wm_poi_id,price__total__skew_by__uuid#wm_poi_id,price__order_id__count_by__uuid#wm_poi_id,dt
0,-7171747965525239881,294039,0.0,0.0,NaN,NaN,1,32.000000,32.0,32.0,32.0,NaN,NaN,1,2017-11-16
1,03B0A314A60BE0F419B4EA7491E9CF969C200974751B68...,497258,0.0,0.0,NaN,NaN,1,50.000000,50.0,50.0,50.0,NaN,NaN,1,2017-11-16
2,04BA099C6A95D66D51EEA7CEE23A20CD177B868459B6CC...,2296536,0.0,0.0,NaN,NaN,1,11.000000,11.0,11.0,11.0,NaN,NaN,1,2017-11-16
3,0884617FA79D122ACB81109736EA73E52EDA3FFD723031...,2407874,1.0,1.0,0.000000,0.0,6,15.283333,14.6,20.3,12.0,3.101236,0.786559,6,2017-11-16
4,09FE3DD5DC5FCDE7EC87D9BC9C51AC27410B7014F7F2B6...,3204360,0.5,0.5,0.707107,NaN,2,12.800000,12.8,13.8,11.8,1.414214,NaN,2,2017-11-16


In [114]:
poiuuid_fea2.head()

,uuid,wm_poi_id,uuidwm_poi_id_G_dt_days_since_last_order,dt
0,-7171747965525239881,294039,15.0,2017-11-16
1,03B0A314A60BE0F419B4EA7491E9CF969C200974751B68...,497258,15.0,2017-11-16
2,04BA099C6A95D66D51EEA7CEE23A20CD177B868459B6CC...,2296536,15.0,2017-11-16
3,0884617FA79D122ACB81109736EA73E52EDA3FFD723031...,2407874,1.0,2017-11-16
4,09FE3DD5DC5FCDE7EC87D9BC9C51AC27410B7014F7F2B6...,3204360,12.0,2017-11-16


#### feature to train 

In [65]:
x

,uuid,uuid_G_dt_days_since_last_order,dt
0,-7171747965525239881,14.0,2017-11-16
1,03B0A314A60BE0F419B4EA7491E9CF969C200974751B68...,15.0,2017-11-16
2,04BA099C6A95D66D51EEA7CEE23A20CD177B868459B6CC...,1.0,2017-11-16
3,0884617FA79D122ACB81109736EA73E52EDA3FFD723031...,1.0,2017-11-16
4,09FE3DD5DC5FCDE7EC87D9BC9C51AC27410B7014F7F2B6...,9.0,2017-11-16


In [66]:
train.dt.nunique()

10

In [157]:
to_train = train.merge(uuid_fea,on=['uuid','dt'],how='left').\
merge(uuid_fea2,on=['uuid','dt'],how='left').\
merge(poiuuid_fea,on=['uuid','wm_poi_id','dt'],how='left').\
merge(poiuuid_fea2,on=['uuid','wm_poi_id','dt'],how='left').\
merge(uuidtagid_fea,on=['uuid','tag_id','dt'],how='left').\
merge(poi_fea,on=['wm_poi_id','dt'],how='left')



to_test = test.merge(uuid_fea,on=['uuid','dt'],how='left').\
merge(uuid_fea2,on=['uuid','dt'],how='left').\
merge(poiuuid_fea,on=['uuid','wm_poi_id','dt'],how='left').\
merge(poiuuid_fea2,on=['uuid','wm_poi_id','dt'],how='left').\
merge(uuidtagid_fea,on=['uuid','tag_id','dt'],how='left').\
merge(poi_fea,on=['wm_poi_id','dt'],how='left')

In [158]:
to_train.head()

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt,___reorder__mean_by__uuid,___reorder__median_by__uuid,___reorder__std_by__uuid,___reorder__skew_by__uuid,___order_id__count_by__uuid,price__total__mean_by__uuid,price__total__median_by__uuid,price__total__max_by__uuid,price__total__min_by__uuid,price__total__std_by__uuid,price__total__skew_by__uuid,price__order_id__count_by__uuid,reorder_price__total__mean_by__uuid,reorder_price__total__median_by__uuid,reorder_price__total__max_by__uuid,reorder_price__total__min_by__uuid,reorder_price__total__std_by__uuid,reorder_price__total__skew_by__uuid,reorder_price__order_id__count_by__uuid,uuid_G_dt_days_since_last_order,___reorder__mean_by__uuid#wm_poi_id,___reorder__median_by__uuid#wm_poi_id,___reorder__std_by__uuid#wm_poi_id,___reorder__skew_by__uuid#wm_poi_id,___order_id__count_by__uuid#wm_poi_id,price__total__mean_by__uuid#wm_poi_id,price__total__median_by__uuid#wm_poi_id,price__total__max_by__uuid#wm_poi_id,price__total__min_by__uuid#wm_poi_id,price__total__std_by__uuid#wm_poi_id,price__total__skew_by__uuid#wm_poi_id,price__order_id__count_by__uuid#wm_poi_id,uuidwm_poi_id_G_dt_days_since_last_order,___reorder__mean_by__uuid#tag_id,___reorder__median_by__uuid#tag_id,___reorder__std_by__uuid#tag_id,___reorder__skew_by__uuid#tag_id,___order_id__count_by__uuid#tag_id,___reorder__mean_by__wm_poi_id,___reorder__median_by__wm_poi_id,___reorder__std_by__wm_poi_id,___reorder__skew_by__wm_poi_id,___order_id__count_by__wm_poi_id,price__total__mean_by__wm_poi_id,price__total__median_by__wm_poi_id,price__total__max_by__wm_poi_id,price__total__min_by__wm_poi_id,price__total__std_by__wm_poi_id,price__total__skew_by__wm_poi_id,price__order_id__count_by__wm_poi_id,reorder_price__total__mean_by__wm_poi_id,reorder_price__total__median_by__wm_poi_id,reorder_price__total__max_by__wm_poi_id,reorder_price__total__min_by__wm_poi_id,reorder_price__total__std_by__wm_poi_id,reorder_price__total__skew_by__wm_poi_id,reorder_price__order_id__count_by__wm_poi_id
0,-3090531316587891587,635554,3.0,1510829829,69.0,99.0,121443283,31236343,0,0.214286,30.0,106.0,18.0,0.000000,0,11931.000000,7800.000002,266.0,"0,1

In [162]:
poi_features =[ x for x in poi_fea.columns.tolist() if 'by' in x ]
uuid_features =[ x for x in uuid_fea.columns.tolist() if 'by' in x ]
uuid_features2 =[ x for x in uuid_fea2.columns.tolist() if 'G' in x ]
poiuuid_features = [ x for x in poiuuid_fea.columns.tolist() if 'by' in x ]
poiuuid_features2 = [ x for x in poiuuid_fea2.columns.tolist() if 'G' in x ]
uuidtagid_features = [ x for x in uuidtagid_fea.columns.tolist() if 'by' in x ]

In [135]:
poiuuid_fea2.head()

,uuid,wm_poi_id,uuidwm_poi_id_G_dt_days_since_last_order,dt
0,-7171747965525239881,294039,15.0,2017-11-16
1,03B0A314A60BE0F419B4EA7491E9CF969C200974751B68...,497258,15.0,2017-11-16
2,04BA099C6A95D66D51EEA7CEE23A20CD177B868459B6CC...,2296536,15.0,2017-11-16
3,0884617FA79D122ACB81109736EA73E52EDA3FFD723031...,2407874,1.0,2017-11-16
4,09FE3DD5DC5FCDE7EC87D9BC9C51AC27410B7014F7F2B6...,3204360,12.0,2017-11-16


In [138]:
uuid2_features

['uuid_G_dt_days_since_last_order']

In [168]:
poi_features

['___reorder__mean_by__wm_poi_id',
 '___reorder__median_by__wm_poi_id',
 '___reorder__std_by__wm_poi_id',
 '___reorder__skew_by__wm_poi_id',
 '___order_id__count_by__wm_poi_id',
 'price__total__mean_by__wm_poi_id',
 'price__total__median_by__wm_poi_id',
 'price__total__max_by__wm_poi_id',
 'price__total__min_by__wm_poi_id',
 'price__total__std_by__wm_poi_id',
 'price__total__skew_by__wm_poi_id',
 'price__order_id__count_by__wm_poi_id',
 'reorder_price__total__mean_by__wm_poi_id',
 'reorder_price__total__median_by__wm_poi_id',
 'reorder_price__total__max_by__wm_poi_id',
 'reorder_price__total__min_by__wm_poi_id',
 'reorder_price__total__std_by__wm_poi_id',
 'reorder_price__total__skew_by__wm_poi_id',
 'reorder_price__order_id__count_by__wm_poi_id']

In [169]:
poi_features,uuid_features,uuid_features2,poiuuid_features,poiuuid_features2,uuidtagid_features

(['___reorder__mean_by__wm_poi_id',
  '___reorder__median_by__wm_poi_id',
  '___reorder__std_by__wm_poi_id',
  '___reorder__skew_by__wm_poi_id',
  '___order_id__count_by__wm_poi_id',
  'price__total__mean_by__wm_poi_id',
  'price__total__median_by__wm_poi_id',
  'price__total__max_by__wm_poi_id',
  'price__total__min_by__wm_poi_id',
  'price__total__std_by__wm_poi_id',
  'price__total__skew_by__wm_poi_id',
  'price__order_id__count_by__wm_poi_id',
  'reorder_price__total__mean_by__wm_poi_id',
  'reorder_price__total__median_by__wm_poi_id',
  'reorder_price__total__max_by__wm_poi_id',
  'reorder_price__total__min_by__wm_poi_id',
  'reorder_price__total__std_by__wm_poi_id',
  'reorder_price__total__skew_by__wm_poi_id',
  'reorder_price__order_id__count_by__wm_poi_id'],
 ['___reorder__mean_by__uuid',
  '___reorder__median_by__uuid',
  '___reorder__std_by__uuid',
  '___reorder__skew_by__uuid',
  '___order_id__count_by__uuid',
  'price__total__mean_by__uuid',
  'price__total__median_by__uui

In [192]:
features = basic_features  +\ 
poi_features +\
uuid_features +\
#uuid_features2 +\
# poiuuid_features   +\
# poiuuid_features2 +\
# uuidtagid_features
# [x for x in basic_features if "reorder" not in x ]+uuidtagid_features
#['reorder_gap_mean', 'order_gap_mean']

In [177]:
print len(features),features

122 ['area_30day', 'pv_ctr_1day', 'neg_delivery_comment_rate', 'order_cnt_12hr', 'op_time_30day', 'click_tag_pref_15day', 'poi_reorder_counts', 'user_reorder_count', 'food_comment_cnt', 'pv_cvr_7day', 'pos_delivery_comment_rate', 'service_fee_rate_7day', 'comment_3star', 'order_tag_pref_15day', 'comment_4star', 'pic_comment_cnt', 'comment_5star_rate', 'month_original_price', 'order_cnt_14hr', 'pv_ctr_7day', 'pos_comment_rate', 'order_cnt_10hr', 'order_cnt_20hr', 'avg_price_month', 'order_cnt_21hr', 'order_cnt_17hr', 'poi_reorder_rate', 'pv_cxr_1day', 'comment_uv', 'month_order_cnt', 'order_cnt_22hr', 'order_cnt_6hr', 'submit_tag_pref_7day', 'order_cnt_23hr', 'pv_cxr_15day', 'uv_cvr_1day', 'original_price_1day', 'comment_1star', 'order_cnt_19hr', 'order_tag_pref_30day', 'dp_score', 'dp_avg_price', 'order_cnt_1day', 'original_price_7day', 'order_cnt_2hr', 'poi_one_bought', 'total_price_7day', 'uv_cvr_7day', 'neg_comment_rate', 'food_comment_rate', 'total_price_1day', 'user_poi_click_deca

#### find best

In [199]:
#uuid_features2 +\
# poiuuid_features   +\
# poiuuid_features2 +\
# uuidtagid_features
# [x for x in basic_features if "reorder" not in x ]+uuidtagid_features
#['reorder_gap_mean', 'order_gap_mean']

for i in [[],poi_features,uuid_features2,uuid_features,poiuuid_features,poiuuid_features2,uuidtagid_features]:
#     print type(basic_features),type(i)
    features = basic_features + i
    print "add features,"+",".join(i)
    X_train = to_train[features]
    y_train = to_train[label]

    X_val = to_test[features]
    y_val = to_test[label]


    X_train.shape,y_train.shape,X_val.shape

    import lightgbm as lgb
    MAX_ROUNDS = 500

    params = {'num_leaves': 100, 'task': 'train', 'verbose': 1, 'learning_rate': 0.1,
              'min_data_in_leaf': 100,'objective': 'binary', 'boosting_type': 'gbdt', 'metric': ['logloss', 'auc'],
              'feature_fraction': 0.3, "bagging_fraction": 0.7, "bagging_freq": 5,'num_threads': 8}
    # cate_fea = [u'dow', u'year', u'month', u'day_of_week', u'holiday_flg',u'date_int']

    dtrain = lgb.Dataset(
            X_train, label=y_train
        )
    dval = lgb.Dataset(
            X_val, label=y_val, reference=dtrain
        )

    bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=30, verbose_eval=30
    #         feval=lgb_rmsle_score
        )
# bst.pr

add features,
Training until validation scores don't improve for 30 rounds.
[30]	training's auc: 0.757263	valid_1's auc: 0.743881
[60]	training's auc: 0.766427	valid_1's auc: 0.747242
[90]	training's auc: 0.773745	valid_1's auc: 0.747954
[120]	training's auc: 0.779954	valid_1's auc: 0.747862
Early stopping, best iteration is:
[93]	training's auc: 0.774311	valid_1's auc: 0.74799
add features,___reorder__mean_by__wm_poi_id,___reorder__median_by__wm_poi_id,___reorder__std_by__wm_poi_id,___reorder__skew_by__wm_poi_id,___order_id__count_by__wm_poi_id,price__total__mean_by__wm_poi_id,price__total__median_by__wm_poi_id,price__total__max_by__wm_poi_id,price__total__min_by__wm_poi_id,price__total__std_by__wm_poi_id,price__total__skew_by__wm_poi_id,price__order_id__count_by__wm_poi_id,reorder_price__total__mean_by__wm_poi_id,reorder_price__total__median_by__wm_poi_id,reorder_price__total__max_by__wm_poi_id,reorder_price__total__min_by__wm_poi_id,reorder_price__total__std_by__wm_poi_id,reorder_pr

### train

#### lgb

In [178]:
X_train = to_train[features]
y_train = to_train[label]

X_val = to_test[features]
y_val = to_test[label]


X_train.shape,y_train.shape,X_val.shape

import lightgbm as lgb
MAX_ROUNDS = 500

params = {'num_leaves': 100, 'task': 'train', 'verbose': 1, 'learning_rate': 0.1,
          'min_data_in_leaf': 100,'objective': 'binary', 'boosting_type': 'gbdt', 'metric': ['logloss', 'auc'],
          'feature_fraction': 0.3, "bagging_fraction": 0.7, "bagging_freq": 5,'num_threads': 8}
# cate_fea = [u'dow', u'year', u'month', u'day_of_week', u'holiday_flg',u'date_int']

dtrain = lgb.Dataset(
        X_train, label=y_train
    )
dval = lgb.Dataset(
        X_val, label=y_val, reference=dtrain
    )

bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=30, verbose_eval=10
#         feval=lgb_rmsle_score
    )
# bst.pr

In [ ]:
uuid feature 下降一些 0.766199

poi feature  没有影响
best [104]	training's auc: 0.778431	valid_1's auc: 0.749747
basic [93]	training's auc: 0.773905	valid_1's auc: 0.748072

[126]	training's auc: 0.774108	valid_1's auc: 0.748384 74856

In [156]:
df = pd.DataFrame({'feature': bst.feature_name(), 'importances': bst.feature_importance()})
df['fscore'] = df['importances'] / df['importances'].sum()

# print(df)
print(df.sort_values('importances', ascending=False).to_string(index=False))
print  "-" * 20 + " features" + "-" * 20
print [x.strip() for x in df.sort_values('importances', ascending=False).feature.to_string(index=False).split("\n")]

feature  importances    fscore
                         poi_reorder_rate          177  0.067506
                    user_poi_submit_decay          155  0.059115
 uuidwm_poi_id_G_dt_days_since_last_order           77  0.029367
                     click_tag_pref_30day           46  0.017544
     price__total__std_by__uuid#wm_poi_id           44  0.016781
                     user_poi_click_decay           43  0.016400
    ___order_id__count_by__uuid#wm_poi_id           38  0.014493
                       user_reorder_count           37  0.014111
                          avg_price_month           37  0.014111
                       order_cnt_increase           35  0.013349
                    submit_tag_pref_15day           35  0.013349
              price__total__skew_by__uuid           31  0.011823
                            op_time_30day           31  0.011823
                             pv_cxr_15day           31  0.011823
            price__total__median_by__uuid           31  0.0

In [ ]:
user_poi_click_decay          446  0.035754
          user_poi_submit_decay          424  0.033991
               poi_reorder_rate          399  0.031987
             user_reorder_count          354  0.028379
           order_tag_pref_15day          319  0.025573
              user_reorder_rate          310  0.024852
           order_tag_pref_30day          281  0.022527
           click_tag_pref_30day          276  0.022126
             poi_reorder_counts          258  0.020683
          submit_tag_pref_30day          255  0.020443
                 poi_one_bought          253  0.020282
           click_tag_pref_15day          250  0.020042
           submit_tag_pref_7day          236  0.018919
            click_tag_pref_7day          208  0.016675
          submit_tag_pref_15day          205  0.016434
                avg_price_month          194  0.015552
discount_rate_all_customer_7day          185  0.014831

# analysis

In [84]:
train[train.uuid=='922379953431986270']

,uuid,wm_poi_id,shipping_fee,order_time,total,original_price,longitude,latitude,reorder,poi_reorder_rate,poi_reorder_counts,poi_one_bought,poi_two_bought,user_reorder_rate,user_reorder_count,month_original_price,month_total_price,month_order_cnt,timescope_original_price,timescope_total_price,timescope_order_cnt,avg_price_month,comment_1star,comment_2star,comment_3star,comment_4star,comment_5star,comment_uv,pos_delivery_comment_rate,neg_delivery_comment_rate,pos_comment_rate,neg_comment_rate,comment_5star_rate,food_comment_cnt,avg_comment_score,avg_delivery_comment_score,avg_food_comment_score,pv_ctr_1day,uv_ctr_1day,pv_cvr_1day,uv_cvr_1day,pv_cxr_1day,uv_cxr_1day,pv_ctr_7day,uv_ctr_7day,pv_cvr_7day,uv_cvr_7day,pv_cxr_7day,uv_cxr_7day,pv_ctr_15day,uv_ctr_15day,pv_cvr_15day,uv_cvr_15day,pv_cxr_15day,uv_cxr_15day,order_cnt_1hr,order_cnt_2hr,order_cnt_3hr,order_cnt_4hr,order_cnt_5hr,order_cnt_6hr,order_cnt_7hr,order_cnt_8hr,order_cnt_9hr,order_cnt_10hr,order_cnt_11hr,order_cnt_12hr,order_cnt_13hr,order_cnt_14hr,order_cnt_15hr,order_cnt_16hr,order_cnt_17hr,order_cnt_18hr,order_cnt_19hr,order_cnt_20hr,order_cnt_21hr,order_cnt_22hr,order_cnt_23hr,order_cnt_24hr,order_cnt_1day,order_cnt_7day,order_cnt_14day,original_price_1day,original_price_7day,total_price_1day,total_price_7day,service_fee_1day,service_fee_7day,service_fee_rate_1day,service_fee_rate_7day,order_cnt_increase,pic_comment_cnt,food_comment_rate,pic_comment_rate,dp_avg_price,tag_id,dp_score,discount_rate_new_customer_7day,discount_rate_all_customer_7day,distance_30day,area_30day,op_time_30day,user_poi_click_decay,user_poi_submit_decay,user_poi_order_decay,click_tag_pref_3day,click_tag_pref_7day,click_tag_pref_15day,click_tag_pref_30day,submit_tag_pref_3day,submit_tag_pref_7day,submit_tag_pref_15day,submit_tag_pref_30day,order_tag_pref_3day,order_tag_pref_7day,order_tag_pref_15day,order_tag_pref_30day,dt,dow,dom
71,922379953431986270,1694208,2.0,1509338539,20.00,20.0,112567201,33032867,1,0.287037,186.0,427.0,136.0,0.666667,2,36376.0,33330.059825,1368.0,"0,941.939712,134.3196928,436.6704128,191.38496","0,874.8989858713867,119.95735383222657,397.748...","0,38.4326144,5.8749952,18.2089216,7.0756864",24.364079,2.0,19.0,57.0,68.0,464.0,479.0,0.990164,0.006557,0.872131,0.034426,0.760656,610.0,4.595082,4.942623,4.595082,0.060164,0.082131,0.230088,0.250000,0.015187,0.023307,0.065471,0.100000,0.213714,0.249619,0.014154,0.025348,0.075920,0.108469,0.211661,0.243823,0.016148,0.026620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,75.0,65.0,18.0,12.0,9.0,10.0,7.0,23.0,22.0,24.0,14.0,6.0,0.0,0.0,39.0,293.0,634.0,882.5,7359.5,825.929996,6711.759950,241.5,1741.0,0.133196,0.112685,-48.0,13.0,0.445906,0.009503,0.0,99,73.0,0.626458,0.927659,0.570423,12378026.31,42961.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-30,0,30
2918,922379953431986270,1449951,2.0,1509598045,20.00,22.0,112567201,33032867,0,0.245333,92.0,276.0,56.0,0.666667,2,32369.0,24062.799817,969.0,"0,804.1768959999999,45.0368,289.93172480000004...","0,585.1675808740234,38.4703998046875,220.96766...","0,23.1077888,1.7216,10.7105792,0.4096",24.832611,6.0,9.0,38.0,52.0,285.0,303.0,0.997436,0.000000,0.864103,0.038462,0.730769,390.0,4.541026,4.961538,4.541026,0.054846,0.071633,0.108434,0.150000,0.006762,0.012894,0.063215,0.091753,0.163062,0.195604,0.010482,0.018351,0.069550,0.096955,0.184170,0.216769,0.012907,0.021234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,47.0,62.0,23.0,9.0,0.0,0.0,5.0,21.0,22.0,7.0,7.0,0.0,0.0,0.0,24.0,208.0,439.0,819.5,6725.5,630.600000,5044.899960,155.5,1206.5,0.123090,0.124393,-23.0,12.0,0.402477,0.012384,0.0,99,0.0,0.565036,0.771034,0.659829,15605197.78,26446.862069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-11-02,3,2
14656,922379953431986270,1694208,2.0,1510295144,25.90,28.0,112567201,33032867,1,0.287037,186.0,427.0,136.0,0.666667,2,34350.0,31490.469803,1354.0,"0,783.5709184000001,112.40064000000001,626.683...","0,700.0509759514894,103.53149157138674,581.053...","0,3

In [91]:
sort_data = data.sort_values('order_time')

In [92]:
sort_data.to_csv(inp_path+'ueq_sort_2_m',index=False)

# base func

In [1]:
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

In [2]:
def ka_create_groupby_features(df, group_columns_list, method_dict, add_to_original_data=False, verbose=1, verbose_detail="create stats features",):
    '''Create statistical columns, group by [N columns] and compute stats on [N column]
       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       method_dict: python dictionary
          Dictionay used to create stats variables
       add_to_original_data: boolean
          only keep stats or add stats variable to raw data
       verbose: int
          1 return tick_tock info 0 do not return any info
       Return
       ------
       new pandas dataframe with original columns and new added columns
       Example
       -------
       ka_add_groupby_features(data
                               ,['class']
                               ,{'before': ['count','mean']})
       Update
       ------
       2017/09/26: pandas 0.20.3 has deprecate using just a dict to rename and create stat variables
       ,so I add another parameter method_list to fix this warning.
       2017/9/27: add verbose_detail parameter, let user specify infos they want print.
       2017/10/8: generate column names automatic
    '''
    with tick_tock(verbose_detail, verbose):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(method_dict)
        print the_stats.columns.ravel()
        the_stats.columns = [''.join(group_columns_list) + '_G_' +"_".join(x) for x in the_stats.columns.ravel()]
        the_stats.reset_index(inplace=True)
        if not add_to_original_data:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

In [3]:
import gc
import time
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import argparse
from utils import nice_method as nm
import pandas as pd
# import sklearn.metrics
# from sklearn.metrics import f1_score, roc_auc_score
# from sklearn.model_selection import train_test_split
import lightgbm as lgb
import lgbm_cv as lc
import numpy as np
import operator

try:
    import matplotlib.pyplot as plt
except ImportError:
    raise ImportError('You need to install matplotlib for plot_example.py.')

In [4]:
def init_arguments():
    parser = argparse.ArgumentParser()

    parser.add_argument('--label', type=str, dest='label',
                        help="data")

    parser.add_argument('--train', type=str, dest='train_path',
                        help="data")
    parser.add_argument('--test', type=str, dest='test_path',
                        help="data")
    parser.add_argument('--round', type=int, dest='round',
                        help="data", default=500)

    parser.add_argument('--output', type=str, dest='out_path',
                        help="data")
    return parser.parse_args()


import random


def deco_data(feas, is_train=False):
    feas.columns = map(lambda x: x.replace("waimai_ad_join_reorder_v8.", ""), feas.columns.tolist())
    # rs = nm.ka_add_groupby_features_1_vs_n(feas, ["uuid"], {"reorder": {"sum": "sum", "count": "count"}})
    if is_train:
        # with_reorder = pd.DataFrame(rs[rs['sum'] > 0]["uuid"])
        # feas = pd.merge(feas, with_reorder, how="inner", on="uuid")
        b_len = len(feas)
        # feas = feas[feas['user_reorder_rate'] > 0]
        print "before filter , lenght is {}, afther filter length is {}".format(b_len, len(feas))

    # feas['poi_reorder_rate'] = feas["user_reorder_rate"]*3 + feas["poi_reorder_rate"]
    # feas['poi_reorder_counts'] = feas["user_reorder_count"]*10 + feas["poi_reorder_counts"]
    # feas['poi_reorder_rate'] = feas["poi_reorder_rate"].apply(lambda x: round(x, 3))

    return feas

In [5]:
dates={}
def lookup(s):
    global dates
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    for date in s.unique():
#         print date
#         print pd.to_datetime(date)
        dates[date] = dates[date] if date in dates else pd.to_datetime(str(date))
#     dates = {date: pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [6]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [7]:
pd.to_datetime(20171016)

Timestamp('1970-01-01 00:00:00.020171016')

In [8]:
lookup(data['dt'].head())

NameError: name 'data' is not defined

In [9]:
dates

{}

In [10]:
def get_stats_target(df, group_column, target_column_list, tgt_stats=['mean', 'median', 'max', 'min', 'std','skew'],
                     prefix="hehe", drop_count=True, drop_raw_col=True, filter_count=10):
    df_old = df.copy()
    grouped = df_old.groupby(group_column)
    grouped.order_id.count()
    rs = []
    for target_column in target_column_list:
        def gene_col_names(stat):
            return '{}__{}__{}_by__{}'.format(prefix, target_column, stat, "#".join(group_column))

        the_stats = grouped[target_column].agg(tgt_stats).reset_index()
        # print the_stats.columns
        the_stats.columns = group_column + map(gene_col_names, tgt_stats)
        if drop_raw_col:
            the_stats.drop(group_column, axis=1, inplace=True)
        rs.append(the_stats)

    count = grouped["order_id"].agg(['count']).reset_index()
    count_col_name = '{}__{}__{}_by__{}'.format(prefix, "order_id", "count", "#".join(group_column))
    count.columns = group_column + [count_col_name]
    agg_rs = pd.concat(rs + [count], axis=1)
    af_flt = agg_rs[agg_rs[count_col_name] >= filter_count]

    print "prefix is {},group column is {},agg total count is {}; after filter , agg total count is {}, filter ratio is {} %". \
        format(prefix, "#".join(group_column), len(agg_rs), len(af_flt),
               (1 - len(af_flt) / (len(agg_rs) + 0.01)) * 100)

    if drop_count:
        return af_flt[[x for x in af_flt.columns if "count" not in x]]
    else:
        return af_flt

# test

In [104]:
import numpy as np
import pandas as pd

N = 100
data = pd.DataFrame({
    'type': np.random.randint(10, size=N),
    'status': np.random.randint(10, size=N),
    'name': np.random.randint(10, size=N),
    'value': np.random.randint(10, size=N),
})

reading = np.random.random(10,)

data = data.groupby(['type', 'status', 'name'])['value'].agg(
    [('one',  np.mean), 
    ('two', lambda value: 100* ((value>32).sum() / reading.mean())), 
    ('test2', lambda value: 100* ((value > 45).sum() / value.mean())),
    ("t3",lambda value:type(value))])
print(data)

                  one  two  test2                                   t3
type status name                                                      
0    0      4     5.0    0    0.0  <class 'pandas.core.series.Series'>
            5     1.0    0    0.0  <class 'pandas.core.series.Series'>
     1      8     3.0    0    0.0  <class 'pandas.core.series.Series'>
     2      3     0.0    0    NaN  <class 'pandas.core.series.Series'>
     3      2     9.0    0    0.0  <class 'pandas.core.series.Series'>
            7     8.0    0    0.0  <class 'pandas.core.series.Series'>
     5      3     8.0    0    0.0  <class 'pandas.core.series.Series'>
     8      7     2.0    0    0.0  <class 'pandas.core.series.Series'>
            9     2.0    0    0.0  <class 'pandas.core.series.Series'>
1    0      8     6.0    0    0.0  <class 'pandas.core.series.Series'>
     1      0     7.0    0    0.0  <class 'pandas.core.series.Series'>
     3      6     1.0    0    0.0  <class 'pandas.core.series.Series'>
     4

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in divide
